### 宣告這次會用到的函式庫

In [29]:
from math import floor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller_fix
from time import sleep
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from lxml import etree
import time

chromedriver_autoinstaller_fix.install()

'c:\\Users\\kao91\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\chromedriver_autoinstaller_fix\\119\\chromedriver.exe'

### 啟動webdriver以及開啟的網址

### 預設欄位名稱

### 建立取得資料的函式

In [74]:
driver = webdriver.Chrome()

url = "https://www4.inservice.edu.tw/script/IndexQuery.aspx?school=10"
html = driver.get(url)

sp = soup(driver.page_source, "lxml")

data_column_name = ["課程代碼", "研習名稱", "性質", "課程開始日期", "課程結束日期", "辦理研習單位", "開課地點"]


def get_data():
    data = []
    row = []
    count = 0
    sp = soup(driver.page_source, "html.parser")

    # Check if the tbody element with class "table_bottom" exists
    table = sp.find("table", class_="csscts")
    if table:
        tr_tags = table.find_all("tr")
        for tr_tag in tr_tags:
            td_tags = tr_tag.find_all("td", class_="wide-only")
            for td_tag in td_tags:
                row.append(td_tag.text)
                count += 1
                if count % 8 == 0:
                    data.append(row[1:])
                    row = []
    return data


def click_next_page_button():
    time.sleep(3)
    next_button = driver.find_element(By.ID, "ImageButton8")
    next_button.click()
    time.sleep(2)


df_list = []
sp = soup(driver.page_source, "html.parser")
total_page = sp.find(id="lbl_Allpage")
total_page = total_page.text
now_page = 1
while True:
    time.sleep(2)
    df_list.extend(get_data())
    if now_page == int(total_page) - 1:
        print("This is the last page ( •̀ ω •́ )✧")
        break
    else:
        click_next_page_button()
        now_page += 1

df = pd.DataFrame(df_list[7:-1], columns=data_column_name)

df.to_csv("教師研習場次.csv", index=False, encoding="utf-8") 
driver.quit()

This is the last page ( •̀ ω •́ )✧
